# Standard CCC and NDR thresholds (CCC>=0.85, NDR>=0.90)

In [ ]:
import pandas as pd

file_path = r"\PATH.xlsx"
data = pd.ExcelFile(file_path).parse('Sheet1')

data.columns = data.columns.str.strip()  # Clean column names of leading/trailing spaces

# thresholds
ccc_threshold = 0.85
ndr_threshold = 0.9

stability_cols = {
    'PFPE Intrasession': ['PFPE intrasession CCC', 'PFPE intrasession NDR'],
    'PFPE Intersession': ['PFPE intersession CCC', 'PFPE intersession NDR'],
    'PFCE Intrasession': ['PFCE intrasession CCC', 'PFCE intrasession NDR'],
    'PFCE Intersession': ['PFCE intersession CCC', 'PFCE intersession NDR'],
    'In Vivo': ['In vivo intrasession CCC', 'In vivo intrasession NDR']
}

data['Feature Class'] = data['Feature'].apply(
    lambda x: 'Shape' if 'shape' in x.lower() else
              'First Order' if any(keyword in x.lower() for keyword in ['firstorder', 'intensity']) else
              'Texture'
)

def calculate_stable_features(data, ccc_col, ndr_col):
    return data[(data[ccc_col] >= ccc_threshold) & (data[ndr_col] >= ndr_threshold)]

scenario_results = {}
for scenario, cols in stability_cols.items():
    ccc_col, ndr_col = cols
    stable_features = calculate_stable_features(data, ccc_col, ndr_col)
    scenario_results[scenario] = {
        'Texture Stable Features': stable_features['Feature Class'].value_counts().get('Texture', 0),
        'First Order Stable Features': stable_features['Feature Class'].value_counts().get('First Order', 0),
        'Shape Stable Features': stable_features['Feature Class'].value_counts().get('Shape', 0),
        'Total Stable Features': len(stable_features)
    }

# Add total feature counts to the scenario-specific stability analysis
total_features = len(data)
total_by_class = data['Feature Class'].value_counts()

df_scenario_stability = pd.DataFrame({
    scenario: {
        'Total Texture Features': total_by_class.get('Texture', 0),
        'Texture Stable Features': scenario_results[scenario]['Texture Stable Features'],
        'Total First Order Features': total_by_class.get('First Order', 0),
        'First Order Stable Features': scenario_results[scenario]['First Order Stable Features'],
        'Total Shape Features': total_by_class.get('Shape', 0),
        'Shape Stable Features': scenario_results[scenario]['Shape Stable Features'],
        'Total Stable Features': scenario_results[scenario]['Total Stable Features']
    } for scenario in scenario_results
}).transpose()

df_scenario_stability['Total Features'] = total_features

# Cross-scenario stability analysis
pfpe_intra = set(calculate_stable_features(data, *stability_cols['PFPE Intrasession'])['Feature'])
pfpe_inter = set(calculate_stable_features(data, *stability_cols['PFPE Intersession'])['Feature'])
pfce_intra = set(calculate_stable_features(data, *stability_cols['PFCE Intrasession'])['Feature'])
pfce_inter = set(calculate_stable_features(data, *stability_cols['PFCE Intersession'])['Feature'])
in_vivo = set(calculate_stable_features(data, *stability_cols['In Vivo'])['Feature'])

shared_pfpe = pfpe_intra & pfpe_inter
shared_pfce = pfce_intra & pfce_inter
exclusive_pfpe_intra = pfpe_intra - shared_pfpe
exclusive_pfpe_inter = pfpe_inter - shared_pfpe
exclusive_pfce_intra = pfce_intra - shared_pfce
exclusive_pfce_inter = pfce_inter - shared_pfce
shared_all = pfpe_intra & pfpe_inter & pfce_intra & pfce_inter & in_vivo
shared_pfpe_pfce = shared_pfce & shared_pfpe

# Create a structured DataFrame for cross-scenario stability
df_cross_scenario_stability = pd.DataFrame([
    {'Category': 'Exclusive PFPE Intrasession', 'Count': len(exclusive_pfpe_intra)},
    {'Category': 'Exclusive PFPE Intersession', 'Count': len(exclusive_pfpe_inter)},
    {'Category': 'Exclusive PFCE Intrasession', 'Count': len(exclusive_pfce_intra)},
    {'Category': 'Exclusive PFCE Intersession', 'Count': len(exclusive_pfce_inter)},
    {'Category': 'Shared Between PFPE Intra & Inter', 'Count': len(shared_pfpe)},
    {'Category': 'Shared Between PFCE Intra & Inter', 'Count': len(shared_pfce)},
    {'Category': 'Shared between PFPE & PFCE', 'Count': len(shared_pfpe_pfce)},
    {'Category': 'Shared Across All Scenarios', 'Count': len(shared_all)}
])

# df_scenario_stability.to_excel('stability_phantoms.xlsx')
# df_cross_scenario_stability.to_excel('stability_phantoms_venn.xlsx')

display(df_scenario_stability)
display(df_cross_scenario_stability)

# Sensitivity testing CCC and NDR thresholds

In [ ]:
# Sensitivity Analysis for CCC + NDR Stability Thresholds
import pandas as pd
import itertools

file_path = r"\PATH.xlsx"
data = pd.read_excel(file_path, sheet_name="Sheet1")
data.columns = data.columns.str.strip()  # Clean column names

data['Feature Class'] = data['Feature'].apply(
    lambda x: 'Shape' if 'shape' in x.lower() else
              'First Order' if any(kw in x.lower() for kw in ['firstorder', 'intensity']) else
              'Texture'
)

stability_cols = {
    'PFPE Intrasession': ['PFPE intrasession CCC', 'PFPE intrasession NDR'],
    'PFPE Intersession': ['PFPE intersession CCC', 'PFPE intersession NDR'],
    'PFCE Intrasession': ['PFCE intrasession CCC', 'PFCE intrasession NDR'],
    'PFCE Intersession': ['PFCE intersession CCC', 'PFCE intersession NDR'],
    'In Vivo': ['In vivo intrasession CCC', 'In vivo intrasession NDR']
}

# Threshold values for sensitivity analysis
ccc_thresholds = [0.75, 0.80, 0.85, 0.90]
ndr_thresholds = [0.85, 0.90, 0.95]
threshold_combos = list(itertools.product(ccc_thresholds, ndr_thresholds))

def calculate_stable_features(data, ccc_col, ndr_col, ccc_thresh, ndr_thresh):
    return data[(data[ccc_col] >= ccc_thresh) & (data[ndr_col] >= ndr_thresh)]

all_sensitivity_results = []

for ccc_thresh, ndr_thresh in threshold_combos:
    for scenario, (ccc_col, ndr_col) in stability_cols.items():
        stable = calculate_stable_features(data, ccc_col, ndr_col, ccc_thresh, ndr_thresh)
        summary = {
            'CCC Threshold': ccc_thresh,
            'NDR Threshold': ndr_thresh,
            'Scenario': scenario,
            'Stable Shape Features': stable['Feature Class'].value_counts().get('Shape', 0),
            'Stable First Order Features': stable['Feature Class'].value_counts().get('First Order', 0),
            'Stable Texture Features': stable['Feature Class'].value_counts().get('Texture', 0),
            'Total Stable Features': len(stable)
        }
        all_sensitivity_results.append(summary)

df_sensitivity = pd.DataFrame(all_sensitivity_results)

df_sensitivity#.to_excel("sensitivity_analysis_stability.xlsx", index=False)

In [ ]:
# Focusing on total stable features only
pivot = df_sensitivity.pivot_table(
    index=['Scenario'],
    columns=['CCC Threshold', 'NDR Threshold'],
    values='Total Stable Features'
)

pivot = pivot.sort_index(axis=1, ascending=[True, True])

# Mark the baseline with asterisk 
pivot.columns = [
    f"{ccc:.2f}, {ndr:.2f}" + ("*" if (ccc == 0.85 and ndr == 0.90) else "")
    for ccc, ndr in pivot.columns
]

pivot#.to_excel("pivoted_sensitivity.xlsx", index=False)